In [2]:
!pip install meteostat

In [5]:
import pandas as pd
from datetime import datetime
from meteostat import Point, Hourly

# Helper function to clean time data
def clean_data(df_path):
    df = pd.read_csv(df_path)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y %I:%M:%S %p')
    return df

# Load existing datasets
actual_prices = clean_data('ActualEnergyPrices2.csv')
day_ahead = clean_data('DayAheadEnergyPrice2.csv')
actual_load = clean_data('ActualLoad2.csv')
forecasted_load = clean_data('ForecastedLoad2.csv')

# Separate Illinois and non-Illinois (HUBs) data
illinois_prices = actual_prices[actual_prices['HUB'] == 'ILLINOIS.HUB'].copy()
other_prices = actual_prices[actual_prices['HUB'] != 'ILLINOIS.HUB'].copy()

# Compute average LMP of surrounding states and add to dataset
avg_other = (
    other_prices.groupby('Date')['LMP']
    .mean()
    .reset_index()
    .rename(columns={'LMP': 'AveragePrice_OtherStates'})
)
merged_df = pd.merge(illinois_prices, avg_other, on='Date', how='left')

# Merge with Day-Ahead, Actual Load, and Forecasted Load
dfs_to_merge = [day_ahead, actual_load, forecasted_load]

for df in dfs_to_merge:
    merged_df = pd.merge(merged_df, df, on='Date', how='left')

# Rename columns (clarity purpose)
merged_df.rename(
    columns={
        'LMP': 'actual_lmp',
        'lmp': 'day_ahead_lmp',
        'Load_x': 'actual_load',
        'Load_y': 'forecasted_load'
    },
    inplace=True
)

# Round timestamps to the hour and drop duplicates
merged_df['Date'] = merged_df['Date'].dt.floor('H')
merged_df = merged_df.drop_duplicates(subset='Date', keep='first')

# Drop unnecessary columns
for col in ['node', 'HUB']:
    if col in merged_df.columns:
        merged_df.drop(columns=[col], inplace=True)



################ MERGING WEATHER DATA (huge clump of code) #################################
merged_df['Date'] = pd.to_datetime(merged_df['Date'])

start_date = merged_df['Date'].min()
end_date = merged_df['Date'].max()

# 3. Define the location (Chicago O'Hare Airport)
# Latitude, Longitude, Altitude for Chicago
location = Point(41.9742, -87.9073, 205)

# 4. Fetch the hourly weather data
print(f"Fetching weather data for Chicago O'Hare from {start_date} to {end_date}...")
weather_data = Hourly(location, start_date, end_date).fetch()

# Select and process weather features
weather_features = weather_data[['temp', 'wspd', 'coco']].copy()
weather_features.rename(columns={'temp': 'temperature_celsius', 'wspd': 'wind_speed_kmh'}, inplace=True)
weather_features['coco'].fillna(method='ffill', inplace=True)

# Define the human-readable map for coco codes
coco_map = {
    1: 'Clear', 2: 'Fair', 3: 'Cloudy', 4: 'Overcast', 5: 'Fog',
    6: 'Freezing_Fog', 7: 'Light_Rain', 8: 'Rain', 9: 'Heavy_Rain',
    10: 'Freezing_Rain', 11: 'Heavy_Freezing_Rain', 12: 'Sleet',
    13: 'Heavy_Sleet', 14: 'Light_Snow', 15: 'Snow', 16: 'Heavy_Snow',
    17: 'Rain_Shower', 18: 'Heavy_Rain_Shower', 19: 'Sleet_Shower',
    20: 'Heavy_Sleet_Shower', 21: 'Snow_Shower', 22: 'Heavy_Snow_Shower',
    23: 'Lightning', 24: 'Hail', 25: 'Thunderstorm',
    26: 'Heavy_Thunderstorm', 27: 'Storm'
}
# Note: I replaced spaces with underscores for cleaner column names.

# Perform One-Hot Encoding for weather columns
coco_dummies = pd.get_dummies(weather_features['coco'], prefix='coco')

# Create a dictionary to map the old column names to the new ones
# e.g., {'coco_1.0': 'coco_Clear', 'coco_7.0': 'coco_Light_Rain'}
new_column_names = {}
for col in coco_dummies.columns:
    # Extract the code number (e.g., '1.0' from 'coco_1.0')
    code_str = col.split('_')[1]
    code_int = int(float(code_str))

    # Find the description in the map, default to the code if not found
    description = coco_map.get(code_int, f'Unknown_{code_int}')
    new_column_names[col] = f'coco_{description}'

# Column adjustment
coco_dummies.rename(columns=new_column_names, inplace=True)
weather_features = weather_features.join(coco_dummies)
weather_features.drop('coco', axis=1, inplace=True)

# Merging weather data - We merge based on the 'Date' column and the index of the weather data
merged_df = pd.merge(merged_df, weather_features, left_on='Date', right_index=True, how='left')

bool_cols = merged_df.select_dtypes(include='bool').columns
merged_df[bool_cols] = merged_df[bool_cols].astype(int)

# Display the first few rows of the final, merged DataFrame
print("\nPreview of the merged DataFrame with weather data:")
print(merged_df.head())

Fetching weather data for Chicago O'Hare from 2022-01-01 00:00:00 to 2025-02-05 18:00:00...

Preview of the merged DataFrame with weather data:
                  Date  actual_lmp  AveragePrice_OtherStates  day_ahead_lmp  \
0  2022-01-01 00:00:00       22.46                 25.725714          29.54   
8  2022-01-01 02:00:00       22.54                 23.585714          26.32   
16 2022-01-01 03:00:00       21.82                 22.627143          23.81   
24 2022-01-01 04:00:00       21.43                 22.382857          22.58   
32 2022-01-01 05:00:00       20.69                 21.810000          22.60   

    actual_load  forecasted_load  temperature_celsius  wind_speed_kmh  \
0       67114.0            68692                  5.6            14.8   
8       63696.0            62585                  5.0            14.8   
16      62627.0            61348                  4.4            24.1   
24      62217.0            60685                  4.4            18.4   
32      61962.0 

The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




In [ ]:
# Export results
merged_df.to_csv('merged_data4.csv', index=False)
merged_df.to_pickle('merged_data.pkl')